In [4]:
import os
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, ForeignKey, event

# === 1. Supprimer ancienne base ===
db_path = "nutriprofil_final.db"
if os.path.exists(db_path):
    os.remove(db_path)

# === 2. Créer l'engine ET activer les foreign keys ===
engine = create_engine(f"sqlite:///{db_path}")

@event.listens_for(engine, "connect")
def enforce_foreign_keys(dbapi_connection, connection_record):
    cursor = dbapi_connection.cursor()
    cursor.execute("PRAGMA foreign_keys = ON")
    cursor.close()

# === 3. Définir les tables ===
metadata = MetaData()

dim_time = Table(
    'dim_time', metadata,
    Column('year_id', Integer, primary_key=True)
)

dim_family = Table(
    'dim_family', metadata,
    Column('family_id', Integer, primary_key=True, autoincrement=True),
    Column('family_name', String, unique=True, nullable=False)
)

dim_food = Table(
    'dim_food', metadata,
    Column('food_id', Integer, primary_key=True, autoincrement=True),
    Column('name', String, nullable=False),
    Column('family_id', Integer, ForeignKey('dim_family.family_id')),
    Column('glucides', Float),
    Column('gras', Float),
    Column('ig', Float),
    Column('cg', Float),
    Column('proteines', Float)
)

fact_consumption = Table(
    'fact_consumption', metadata,
    Column('year_id', Integer, ForeignKey('dim_time.year_id'), primary_key=True),
    Column('family_id', Integer, ForeignKey('dim_family.family_id'), primary_key=True),
    Column('value', Float, nullable=False)
)

fact_health = Table(
    'fact_health', metadata,
    Column('year_id', Integer, ForeignKey('dim_time.year_id'), primary_key=True),
    Column('age_group_name', String, primary_key=True),
    Column('sex_name', String, primary_key=True),
    Column('diabete_prev', Float),
    Column('mcv_mort_rate', Float),
    Column('cancercolorectal_mort', Float),
    Column('alzheimer_mort', Float),
    Column('ssbs', Float),
    Column('fiber', Float),
    Column('sodium', Float),
    Column('transfat', Float)
)

food_health_link = Table(
    'food_health_link', metadata,
    Column('link_id', Integer, primary_key=True, autoincrement=True),
    Column('food_id', Integer, ForeignKey('dim_food.food_id')),
    Column('indicateur', String),
    Column('effet', String),
    Column('seuil_description', String),
    Column('direction', String),
    Column('source', String)
)

# === 4. Créer les tables ===
metadata.create_all(engine)

# === 5. Charger les CSV ===
df_cons = pd.read_csv("/Users/coulibalykani/Desktop/Projet 3/projet3/Dataframe/previsions_conso_ML_Lasso.csv")
df_food = pd.read_csv("/Users/coulibalykani/Desktop/Projet 3/projet3/Dataframe/DF_Aliment_NUTRIPROFIL.csv")
df_health = pd.read_csv("/Users/coulibalykani/Desktop/Projet 3/projet3/Dataframe/france_nutrition_sante_fusionfinal.csv")

# === 6. Nettoyage + insertion ===
if 'Famille de produits' in df_cons.columns:
    df_cons = df_cons.rename(columns={'Famille de produits': 'famille'})

year_cols = df_cons.columns.drop('famille')
years = pd.DataFrame({'year_id': sorted(set(year_cols.astype(int)) | set(df_health['year_id']))})
years.to_sql('dim_time', engine, if_exists='replace', index=False)

df_fam = pd.DataFrame(df_cons['famille'].unique(), columns=['family_name'])
df_fam.to_sql('dim_family', engine, if_exists='replace', index_label='family_id')

dim_fam = pd.read_sql_table('dim_family', engine, index_col='family_id').reset_index()
if 'Famille_regroupee' in df_food.columns:
    df_food = df_food.rename(columns={'Famille_regroupee': 'family_name'})
else:
    raise KeyError("❌ 'Famille_regroupee' non trouvée dans df_food")

df_food = df_food.merge(dim_fam, on='family_name', how='left')
df_food.to_sql('dim_food', engine, if_exists='replace', index_label='food_id')

df_fc = df_cons.melt(id_vars=['famille'], var_name='year_id', value_name='value')
df_fc = df_fc.rename(columns={'famille': 'family_name'})
df_fc['year_id'] = df_fc['year_id'].astype(int)
df_fc = df_fc.merge(dim_fam, on='family_name', how='left')
df_fc[['year_id', 'family_id', 'value']].to_sql('fact_consumption', engine, if_exists='replace', index=False)

df_health.to_sql('fact_health', engine, if_exists='replace', index=False)

food_health_data = pd.DataFrame([
    {
        'food_id': 1,
        'indicateur': 'diabete_prev',
        'effet': '+9% risque de diabète',
        'seuil_description': 'par +10g/j de sucres ajoutés',
        'direction': 'augmentation',
        'source': 'GBD 2019'
    },
    {
        'food_id': 1,
        'indicateur': 'mcv_mort_rate',
        'effet': '+12% mortalité cardiovasculaire',
        'seuil_description': 'par 100ml/j de soda',
        'direction': 'augmentation',
        'source': 'Lancet Study'
    }
])
food_health_data.to_sql('food_health_link', engine, if_exists='replace', index=False)

print("✅ Base SQLite créée avec toutes les relations activées.")


✅ Base SQLite créée avec toutes les relations activées.
